In [100]:
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

In [101]:
# We need to get the following params: import_status date_onset date_confirm report_delay region

In [125]:
import numpy as np
import pandas as pd

In [126]:
data = pd.read_csv('../latest_raw.csv', encoding='utf-8')

In [127]:
data = data[[
    'ID_REGISTRO','FECHA_INGRESO','FECHA_SINTOMAS','FECHA_DEF','ENTIDAD_RES','ENTIDAD_UM','PAIS_ORIGEN','RESULTADO'
]]

In [128]:
print(data.set_index(['ID_REGISTRO']).index.is_unique)

False


In [160]:
dat = data.copy()

In [161]:
#dat.loc[dat['FECHA_INGRESO'].str.startswith('9999'), 'FECHA_INGRESO'] = np.nan
#dat.loc[dat['FECHA_DEF'].str.startswith('9999'), 'FECHA_DEF'] = np.nan
#dat.loc[dat['FECHA_SINTOMAS'].str.startswith('9999'), 'FECHA_SINTOMAS'] = np.nan

In [162]:
dat['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
dat['FECHA_INGRESO'] = dat['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
dat = dat.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [163]:
import janitor

In [164]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20') # dates group by testing site 
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by residence
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'}) # set region

In [165]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-21','2022-12-31') # dates to group by region of residence
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'}) # set region

In [166]:
tbl = pd.concat([tbl_x, tbl_y], sort=False)

In [178]:
tbl = tbl.rename(
    columns = {
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'FECHA_DEF':'Date_Death',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [179]:
geo = pd.read_csv('../data/geo/entidades.csv')

In [180]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [181]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [182]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [183]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Date_Death', 'Region', 'Region_ID', 'Region_Key', 'Origin', 'Status']]

In [184]:
output['Region'] = output['Region'].str.replace('Á','A')
output['Region'] = output['Region'].str.replace('É','E')
output['Region'] = output['Region'].str.replace('Í','I')
output['Region'] = output['Region'].str.replace('Ó','O')
output['Region'] = output['Region'].str.replace('Ú','U')

In [185]:
output['Region'] = output['Region'].str.replace('COAHUILA DE ZARAGOZA','COAHUILA')
output['Region'] = output['Region'].str.replace('MICHOACAN DE OCAMPO','MICHOACAN')
output['Region'] = output['Region'].str.replace('VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ')

In [186]:
output = output.sort_values(['Date_Confirmed', 'Date_Symptoms', 'Region'])

In [187]:
output.Region.unique()

array(['JALISCO', 'QUINTANA ROO', 'CIUDAD DE MEXICO', 'COAHUILA',
       'VERACRUZ', 'BAJA CALIFORNIA', 'DURANGO', 'TABASCO', 'MEXICO',
       'NAYARIT', 'SONORA', 'GUANAJUATO', 'MORELOS', 'TAMAULIPAS',
       'HIDALGO', 'PUEBLA', 'AGUASCALIENTES', 'BAJA CALIFORNIA SUR',
       'YUCATAN', 'MICHOACAN', 'GUERRERO', 'OAXACA', 'SINALOA',
       'NUEVO LEON', 'TLAXCALA', 'CHIHUAHUA', 'ZACATECAS',
       'SAN LUIS POTOSI', 'CHIAPAS', 'QUERETARO', 'CAMPECHE', 'COLIMA'],
      dtype=object)

In [188]:
output.Origin.unique()

array(['Local', 'Alemania', 'Cuba', 'Grecia', 'Nicaragua',
       'Estados Unidos de América', 'Estados Unidos de Am�rica', 'Otro',
       'España', 'Espa�a', 'Colombia', 'Egipto', 'El Salvador', 'Ecuador',
       'Venezuela', 'Haití', 'Hait�', 'República de Honduras',
       'Rep�blica de Honduras', 'Guatemala', 'Francia', 'Bolivia',
       'Dinamarca', 'Japón', 'Jap�n', 'China', 'Perú', 'Per�', 'Camerún',
       'Camer�n', 'Chile', 'Eritrea', 'Canadá', 'Canad�', 'Brasil',
       'Belice', 'Irlanda', 'Costa de Marfil', 'Italia', 'Malasia',
       'Argentina', 'Suecia'], dtype=object)

In [189]:
output

,Date_Confirmed,Date_Symptoms,Date_Death,Region,Region_ID,Region_Key,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
11331,2020-01-06,2020-01-02,9999-99-99,QUINTANA ROO,23,QR,Local,2
22425,2020-01-06,2020-01-03,9999-99-99,CIUDAD DE MEXICO,9,DF,Local,2
14738,2020-01-06,2020-01-03,9999-99-99,COAHUILA,5,CL,Local,2
1,2020-01-06,2020-01-03,9999-99-99,JALISCO,14,JC,Local,2
...,...,...,...,...,...,...,...,...
143627,2020-05-08,2020-05-08,9999-99-99,TAMAULIPAS,28,TS,Local,3
143628,2020-05-08,2020-05-08,9999-99-99,TAMAULIPAS,28,TS,Local,3
77927,2020-05-08,2020-05-08,9999-99-99,VERACRUZ,30,VZ,Local,3
161144,2020-05-08,2020-05-08,9999-99-99,YUCATAN,31,YN,Local,3


### By status

In [61]:
# Latest time series

In [73]:
latest = output.copy()

In [74]:
latest = latest[latest['Status']==1]
latest = latest[latest.Date_Death.str.startswith(('9999'))] # active cases in patients not deceased
latest = latest[['Date_Confirmed','Date_Symptoms','Date_Death','Region','Region_ID','Region_Key','Origin','Status']]

In [76]:
latest.to_csv('../latest.csv', index=False)